In [1]:
#importing libraries
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import holidays
import joblib
import json
from sklearn.utils.discovery import all_estimators
from scipy.stats import randint,uniform


### Data Preprocessing

In [2]:
#reading in
df = pd.read_csv("Sample Dataset.csv")

# Feature Engineering
holiday = holidays.country_holidays("CA", subdiv="ON")
df["Date"] = pd.to_datetime(df["Date"])
df["DateTime"] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Hour'].astype(str), format='%Y-%m-%d %H')
df = df.drop_duplicates(subset=["DateTime"])
df["Holiday"] = (df["DateTime"].apply(lambda x: x in (holiday))).astype(int)
df["Weekday_num"] = (df["DateTime"]).dt.dayofweek
df["Month_num"] = (df["DateTime"]).dt.month
df["Day"] = df["DateTime"].dt.day
df = df.set_index(["DateTime"])


,Date,Weekday,Hour,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,Holiday,Weekday_num,Month_num,Day
DateTime,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01,Friday,0,0.49,14023,-0.3,-5.62,20,-3.18,70,-5.1,99.67,1,4,1,1
2016-01-01 01:00:00,2016-01-01,Friday,1,-1.09,13417,-0.3,-6.30,25,-3.48,68,-5.5,99.63,1,4,1,1
2016-01-01 02:00:00,2016-01-01,Friday,2,-2.41,12968,-0.4,-6.55,26,-3.43,73,-4.7,99.59,1,4,1,1
2016-01-01 12:00:00,2016-01-01,Friday,12,0.00,14215,-2.1,-8.72,23,-4.86,76,-5.8,99.39,1,4,1,1
2016-01-01 13:00:00,2016-01-01,Friday,13,0.00,14443,-2.4,-8.23,19,-5.09,84,-4.7,99.35,1,4,1,1


### Visualization

In [ ]:
#visualizing histogram
for col in df.select_dtypes("number").columns:
    x=df[col]
    fig = go.Figure(data=[go.Histogram(x=x)])
    fig.update_layout(title=col)
    fig.show()

In [ ]:
#visualizing time series
for col in df.select_dtypes("number").columns:
    x=df.index
    y=df[col]
    fig = go.Figure(data=go.Scatter(x=x, y=y))
    fig.update_layout(title=col)
    fig.show()

### Removing outliers

In [11]:
outliers = {"Temperature":{"min":-20},
            "Windchill_Index":{"min":-30},
            "Wind_Speed":{"max":50},
            "Humidex":{"max":45},
            "Dew_Point":{"min":-22}}

In [24]:
#saving outliers file
filename="outliers.json"
with open(filename, 'w') as f:
    json.dump(outliers, f, indent=4)

In [12]:
#removing outliers
for k,v in outliers.items():
    col_name = k
    for lim,num in v.items():
        if lim == "min":
            df = df[df[col_name]>num]
        elif lim == "max":
            df = df[df[col_name]<num]

### Train Test Split

In [14]:
# Shifting dataset
df["Target"] = df["Ontario_Demand"].shift(freq="-24h")



In [15]:
#Train test split
train_df = df[df.index<pd.to_datetime("July 1 2020")].dropna()
test_df = df[df.index>=pd.to_datetime("July 1 2020")].dropna()

X_train = train_df.select_dtypes(exclude=["datetime"]).drop(columns=["Target","Ontario_Demand","Weekday"])
y_train = train_df["Target"]

X_test = test_df.select_dtypes(exclude=["datetime"]).drop(columns=["Target","Ontario_Demand","Weekday"])
y_test = test_df["Target"]



### Modelling

In [16]:
models = all_estimators("regressor")

In [17]:
regressors = ["ElasticNet",
              "ExtraTreesRegressor",
              "GradientBoostingRegressor",
              "HistGradientBoostingRegressor",
              "SVR",
              "Lasso",
              "RandomForestRegressor",
              "DecisionTreeRegressor",
              "Ridge"]
mod_names = []
mod_mae = []
mod_mape = []
for model in regressors:
    for m in models:
        if m[0]==model:
            mod = m[1]
            
    if model in ["SVR","Ridge","Lasso","ElasticNet"]:
        mod = make_pipeline(RobustScaler(),mod())
    else:
        mod = mod()    
    mod.fit(X_train,y_train)
    y_pred = mod.predict(X_test)
    mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)
    mod_names.append(model)
    mod_mae.append(mae)
    mod_mape.append(mape)
    
    

Pipeline(steps=[('robustscaler', RobustScaler()), ('elasticnet', ElasticNet())]) 1633.2244893876914 0.10696550381555894
ExtraTreesRegressor() 703.6518895001195 0.045322555682977717
GradientBoostingRegressor() 785.9593695470651 0.05096910492834491
HistGradientBoostingRegressor() 702.0274196345042 0.04540923163273484
Pipeline(steps=[('robustscaler', RobustScaler()), ('svr', SVR())]) 1597.534545025058 0.10389088537261887


/home/miky/env/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.603e+08, tolerance: 1.429e+07



Pipeline(steps=[('robustscaler', RobustScaler()), ('lasso', Lasso())]) 1350.0711935059796 0.08668690202861375
RandomForestRegressor() 725.1728581679024 0.04669703633323705
DecisionTreeRegressor() 963.2497010284621 0.0615748736141758
Pipeline(steps=[('robustscaler', RobustScaler()), ('ridge', Ridge())]) 1347.345723892686 0.0865246347571774


In [18]:
#saving initial results
res_df = pd.DataFrame({"models":mod_names,"MAE":mod_mae,"MAPE":mod_mape})
res_df.to_csv("res_df.csv",index=False)

In [19]:
params = {"learning_rate":uniform(loc=0.0001,scale=1),
    "max_leaf_nodes":randint(low=1,high=20),
    "max_iter":randint(low=500,high=1500),
    "min_samples_leaf": randint(low=5,high=30),
    "max_bins":randint(low=3,high=255),
    "l2_regularization": uniform(loc=0,scale=2)
          }
tscv = TimeSeriesSplit(n_splits=5)
rscv = RandomizedSearchCV(best_mod,params,n_iter=100,
                          cv=tscv,
                          scoring="neg_mean_absolute_percentage_error",n_jobs=-1,verbose=3)

In [20]:
rscv.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END l2_regularization=1.5440596800704012, learning_rate=0.5727447162386814, max_bins=242, max_iter=747, max_leaf_nodes=2, min_samples_leaf=5;, score=-0.048 total time=   0.4s
[CV 3/5] END l2_regularization=1.5440596800704012, learning_rate=0.5727447162386814, max_bins=242, max_iter=747, max_leaf_nodes=2, min_samples_leaf=5;, score=-0.050 total time=   0.5s
[CV 4/5] END l2_regularization=1.5440596800704012, learning_rate=0.5727447162386814, max_bins=242, max_iter=747, max_leaf_nodes=2, min_samples_leaf=5;, score=-0.045 total time=   0.6s
[CV 1/5] END l2_regularization=1.5440596800704012, learning_rate=0.5727447162386814, max_bins=242, max_iter=747, max_leaf_nodes=2, min_samples_leaf=5;, score=-0.055 total time=   0.7s
[CV 2/5] END l2_regularization=1.3091044357427584, learning_rate=0.6543592842953605, max_bins=206, max_iter=888, max_leaf_nodes=9, min_samples_leaf=22;, score=-0.049 total time=   0.7s
[CV 5/5] END l2_

/home/miky/env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


30 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/home/miky/env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/miky/env/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/miky/env/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/miky/env/lib/python3.11/site-pa

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=HistGradientBoostingRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'l2_regularization': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd91d8925d0>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen objec...
                                        'max_bins': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd91d892950>,
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd91f19ca90>,
                                        'max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd91e8c3b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd91d8922d0>},
                   scoring='neg_mean_absolute_percentage_error', verbose=3)

In [34]:
rscv.best_estimator_

HistGradientBoostingRegressor(l2_regularization=1.4300825251801876,
                              learning_rate=0.07427500203284511, max_bins=122,
                              max_iter=611, max_leaf_nodes=15,
                              min_samples_leaf=26)

In [21]:
y_pred = rscv.predict(X_test)
mean_absolute_error(y_true=y_test,y_pred=y_pred),mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)

(681.2401395086462, 0.04409279801761577)

In [22]:
eval_df = test_df[test_df.index<pd.to_datetime("September 1 2020")].copy()
X_eval = eval_df.select_dtypes(exclude=["datetime"]).drop(columns=["Target","Ontario_Demand","Weekday"])
eval_df["preds"] = rscv.predict(X_eval)
eval_df["abs_error"] = (eval_df["Target"] - eval_df["preds"]).abs()
agg_df = eval_df.groupby(eval_df.index.floor("d")).agg({"abs_error":["mean","count"]})
agg_df.columns = ["mean","count"]
agg_df.sort_values(["mean"])

,mean,count
DateTime,,
2020-07-21,320.095559,24
2020-08-07,365.771352,23
2020-07-15,382.773643,24
2020-08-06,402.382959,21
2020-08-27,431.869870,24
...,...,...
2020-08-25,1411.654497,24
2020-08-09,1434.971870,22
2020-08-04,1459.761256,23


In [23]:
joblib.dump(rscv,"model.pickle")

['model.pickle']